In [1]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
from sklearn import naive_bayes as nb
from sklearn.cross_validation import train_test_split

In [2]:
# raw = pd.read_csv('MicrosoftGestureDataset-RC/data/P1_1_12A_p21.csv',sep=' ',header=None)
# raw=raw.drop(xrange(0,20),axis=0).drop(0,axis=1).drop((xrange(4,80,4)),axis=1).drop(80,axis=1)

In [3]:
column_names=['hip_center_x','hip_center_y','hip_center_z',
              'spine_x','spine_y','spine_z','shoulder_center_x',
              'shoulder_center_y','shoulder_center_z','head_x',
              'head_y','head_z','shoulder_left_x','shoulder_left_y',
             'shoulder_left_z','elbow_left_x','elbow_left_y','elbow_left_z',
             'wrist_left_x','wrist_left_y','wrist_left_z','hand_left_x',
              'hand_left_y','wrist_left_z','shoulder_right_x','shoulder_right_y',
             'shoulder_right_z','elbow_right_x','elbow_right_y','elbow_right_z',
             'wrist_right_x','wrist_right_y','wrist_right_z','hand_right_x',
             'hand_right_y','hand_right_z','hip_left_x','hip_left_y','hip_left_z',
             'knee_left_x','knee_left_y','knee_left_z','ankle_left_x','ankle_left_y',
             'ankle_left_z','foot_left_x','foot_left_y','foot_left_z','hip_right_x',
             'hip_right_y','hip_right_z','knee_right_x','knee_right_y','knee_right_z',
             'ankle_right_x','ankle_right_y','ankle_right_z','foot_right_x','foot_right_y',
             'foot_right_z']



In [4]:
joints = {k: pd.DataFrame([]) for k in column_names }

In [14]:
gestures = pd.DataFrame()
labels = pd.Series()
labels_array=[]
sizes = [1];
i=0
for root, dirs, files in os.walk("MicrosoftGestureDataset-RC/data"):
        for f in files:
            if f.endswith(".csv"):
                temp = pd.read_csv(os.path.join(root,f),sep=' ',header=None).drop(xrange(0,20),axis=0).drop(0,axis=1).drop((xrange(4,80,4)),axis=1).drop(80,axis=1)
                temp
                temp.columns=column_names
                for joint_name in temp.columns:
                    current_timeseries = temp[joint_name]
                    current_timeseries.name = i
                    joints[joint_name] = joints[joint_name].append(current_timeseries)
                labels_array.append([f[f.find('_',3)+1:f.rfind('_')].replace('A',''),i])
                sizes.append(len(temp))
#                 print f[f.find('_',3)+1:f.rfind('_')].replace('A','');
                i=i+1;
                if i>100:
                    break;
print str(i)+" gesture files loaded"

# gestures

101 gesture files loaded


In [43]:
labels_array

[['5', 0],
 ['8', 1],
 ['1', 2],
 ['7', 3],
 ['6', 4],
 ['7', 5],
 ['12', 6],
 ['7', 7],
 ['10', 8],
 ['9', 9],
 ['10', 10],
 ['9', 11],
 ['4', 12],
 ['9', 13],
 ['8', 14],
 ['5', 15],
 ['4', 16],
 ['10', 17],
 ['2', 18],
 ['3', 19],
 ['6', 20],
 ['6', 21],
 ['8', 22],
 ['7', 23],
 ['10', 24],
 ['2', 25],
 ['4', 26],
 ['3', 27],
 ['9', 28],
 ['1', 29],
 ['3', 30],
 ['9', 31],
 ['7', 32],
 ['2', 33],
 ['6', 34],
 ['8', 35],
 ['11', 36],
 ['3', 37],
 ['2', 38],
 ['4', 39],
 ['8', 40],
 ['5', 41],
 ['4', 42],
 ['7', 43],
 ['8', 44],
 ['9', 45],
 ['10', 46],
 ['8', 47],
 ['10', 48],
 ['10', 49],
 ['12', 50],
 ['11', 51],
 ['11', 52],
 ['3', 53],
 ['2', 54],
 ['10', 55],
 ['7', 56],
 ['8', 57],
 ['5', 58],
 ['5', 59],
 ['4', 60],
 ['10', 61],
 ['7', 62],
 ['1', 63],
 ['3', 64],
 ['1', 65],
 ['12', 66],
 ['3', 67],
 ['5', 68],
 ['6', 69],
 ['9', 70],
 ['1', 71],
 ['10', 72],
 ['11', 73],
 ['9', 74],
 ['9', 75],
 ['6', 76],
 ['12', 77],
 ['6', 78],
 ['10', 79],
 ['10', 80],
 ['6', 81],
 ['2',

In [52]:
labels_df = pd.DataFrame(labels_array,columns=['gestureLabel','recordNumber'])

In [6]:
training_set = joints['ankle_right_x'].dropna(axis=1,how='any').reset_index().T.drop('index').T

In [7]:
training_set

,20,21,22,23,24,25,26,27,28,29,...,642,643,644,645,646,647,648,649,650,651
0,-0.002018,-0.002592,-0.004624,-0.010201,-0.014915,-0.023117,-0.028234,-0.027824,-0.028171,-0.032634,...,-0.014135,-0.014084,-0.013989,-0.013782,-0.013837,-0.014397,-0.013689,-0.014039,-0.013411,-0.012955
1,-0.091675,-0.091119,-0.092070,-0.093348,-0.093640,-0.091915,-0.092663,-0.095369,-0.094873,-0.094392,...,-0.072053,-0.071167,-0.070551,-0.072724,-0.073621,-0.073642,-0.076545,-0.080289,-0.087620,-0.088294
2,0.154419,0.153534,0.152757,0.151987,0.150910,0.148484,0.148267,0.148368,0.147970,0.147614,...,0.156199,0.155963,0.156212,0.156733,0.156017,0.156734,0.156869,0.156943,0.157064,0.157240
3,0.161377,0.161711,0.162315,0.163933,0.167538,0.169528,0.171048,0.172361,0.172501,0.172641,...,0.189306,0.188991,0.187297,0.186913,0.186849,0.186687,0.185603,0.185252,0.184430,0.182854
4,0.270996,0.270860,0.270969,0.268342,0.268730,0.268590,0.268758,0.268633,0.268873,0.268862,...,0.269498,0.269330,0.269620,0.269660,0.270387,0.270418,0.270578,0.271063,0.271085,0.271143
5,0.119324,0.119298,0.119335,0.118508,0.118448,0.118286,0.118552,0.117770,0.118255,0.118253,...,0.069664,0.070443,0.072302,0.072244,0.072657,0.072687,0.073182,0.073123,0.073052,0.073182
6,0.196411,0.196662,0.196671,0.196844,0.196773,0.196871,0.196879,0.197051,0.197157,0.197150,...,0.129808,0.130173,0.129976,0.127958,0.126447,0.126399,0.124869,0.123201,0.124705,0.125468
7,0.000000,0.000000,0.000000,0.010574,0.010537,0.010625,0.010829,0.010665,0.011089,0.011178,...,-0.001935,-0.001871,-0.002052,-0.000989,-0.000680,-0.000510,-0.000492,-0.000221,0.000145,0.000237
8,0.151245,0.150564,0.151671,0.152509,0.152989,0.154113,0.153255,0.153028,0.154741,0.153715,...,0.158173,0.158309,0.158730,0.158869,0.158743,0.158060,0.157861,0.157952,0.156772,0.157020
9,-0.196411,-0.195539,-0.195213,-0.194880,-0.194932,-0.196007,-0.197132,-0.197120,-0.197210,-0.197856,...,-0.196037,-0.195739,-0.195395,-0.195565,-0.195092,-0.194722,-0.197014,-0.196724,-0.196801,-0.198409


In [8]:
training_set.shape

(101, 632)

In [9]:
training_set[:101]

,20,21,22,23,24,25,26,27,28,29,...,642,643,644,645,646,647,648,649,650,651
0,-0.002018,-0.002592,-0.004624,-0.010201,-0.014915,-0.023117,-0.028234,-0.027824,-0.028171,-0.032634,...,-0.014135,-0.014084,-0.013989,-0.013782,-0.013837,-0.014397,-0.013689,-0.014039,-0.013411,-0.012955
1,-0.091675,-0.091119,-0.092070,-0.093348,-0.093640,-0.091915,-0.092663,-0.095369,-0.094873,-0.094392,...,-0.072053,-0.071167,-0.070551,-0.072724,-0.073621,-0.073642,-0.076545,-0.080289,-0.087620,-0.088294
2,0.154419,0.153534,0.152757,0.151987,0.150910,0.148484,0.148267,0.148368,0.147970,0.147614,...,0.156199,0.155963,0.156212,0.156733,0.156017,0.156734,0.156869,0.156943,0.157064,0.157240
3,0.161377,0.161711,0.162315,0.163933,0.167538,0.169528,0.171048,0.172361,0.172501,0.172641,...,0.189306,0.188991,0.187297,0.186913,0.186849,0.186687,0.185603,0.185252,0.184430,0.182854
4,0.270996,0.270860,0.270969,0.268342,0.268730,0.268590,0.268758,0.268633,0.268873,0.268862,...,0.269498,0.269330,0.269620,0.269660,0.270387,0.270418,0.270578,0.271063,0.271085,0.271143
5,0.119324,0.119298,0.119335,0.118508,0.118448,0.118286,0.118552,0.117770,0.118255,0.118253,...,0.069664,0.070443,0.072302,0.072244,0.072657,0.072687,0.073182,0.073123,0.073052,0.073182
6,0.196411,0.196662,0.196671,0.196844,0.196773,0.196871,0.196879,0.197051,0.197157,0.197150,...,0.129808,0.130173,0.129976,0.127958,0.126447,0.126399,0.124869,0.123201,0.124705,0.125468
7,0.000000,0.000000,0.000000,0.010574,0.010537,0.010625,0.010829,0.010665,0.011089,0.011178,...,-0.001935,-0.001871,-0.002052,-0.000989,-0.000680,-0.000510,-0.000492,-0.000221,0.000145,0.000237
8,0.151245,0.150564,0.151671,0.152509,0.152989,0.154113,0.153255,0.153028,0.154741,0.153715,...,0.158173,0.158309,0.158730,0.158869,0.158743,0.158060,0.157861,0.157952,0.156772,0.157020
9,-0.196411,-0.195539,-0.195213,-0.194880,-0.194932,-0.196007,-0.197132,-0.197120,-0.197210,-0.197856,...,-0.196037,-0.195739,-0.195395,-0.195565,-0.195092,-0.194722,-0.197014,-0.196724,-0.196801,-0.198409


In [10]:
len(labels_array)

101

In [11]:
classifier = nb.GaussianNB().fit(training_set,labels_array)

In [46]:
results = classifier.predict(training_set)

In [47]:
len(results)

101

In [48]:
len(labels_array)

101

In [54]:
(results==labels_df['gestureLabel']).value_counts()

False    79
True     22
Name: gestureLabel, dtype: int64